本 notebook 是学习 Datewhale 的 llm-universe 所记录的操作过程以及部分笔记，课程地址是：https://github.com/datawhalechina/llm-universe/tree/main

### 使用LLM api


#### temperature 
temperature 参数在语言模型中的作用是通过调整输出概率的分布来控制生成文本的随机性。temperature参数通过缩放 logits（模型的原始输出分数）来改变概率分布。   
调整后的 logits 通过 softmax 函数转化为概率:   
$$ P(w_i) = \frac{\exp(\text{logit}_i / \text{temperature})}{\sum_{j} \exp(\text{logit}_j / \text{temperature})} $$
在 softmax 转换后，较低的温度使得概率分布更集中（更确定），较高的温度使得概率分布更均匀（更多样）。

In [17]:
from zhipuai import ZhipuAI

client = ZhipuAI(api_key="cbd8a94216d99a55d62c3e620e138ecf.4E9KBFzfUmHCwoLD")

def gen_glm_params(prompt):
    ''' 
    构造 GLM 模型请求参数 messages

    请求参数： 
        prompt: 提示词
    '''
    messages = [{"role": "user", "content": prompt}]
    return messages


def get_completion(prompt, model, temperature):
    '''
    获取 GLM 模型调用结果

    请求参数：
        prompt: 对应的提示词
        model: 调用的模型
        temperature: 模型输出的温度系数，控制输出的随机程度，取值范围是 0~1.0，且不能设置为 0。温度系数越低，输出内容越一致。
    '''

    messages = gen_glm_params(prompt)
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature
        # top_k = 0.7
    )
    if len(response.choices) > 0:
        return response.choices[0].message.content
    return "generate answer error"

其中:
 1. messages 的类型是 List, 调用语言模型时，必须填写，将当前对话信息列表作为提示输入给模型按照 {"role": "user", "content": "你好"} 的json 数组形式进行传参； 可能的消息类型包括 System message、User message、Assistant message 和 Tool message。
 2. temperature 和 top_p 都是语言模型生成文本时用来控制输出质量和多样性的参数。有以下区别：
  * ```temperature```参数控制生成文本的随机性。范围是 0 到 1 之间(也有大于 1 的)。
  * ```top_p```参数控制生成文本时考虑的候选词的数量。模型每次计算的结果数量是整个词典的每个词的概率分布，而参数 ```top_p```只保留概率最高的前百分之 p 个候选词的logits，其他设为负无穷。
 3. model 可选的有通用大模型： GLM-4-0520、GLM-4、GLM-4-AirX; 向量大模型：Embedding-2

In [6]:
get_completion("请介绍一下欧式空间？", 'GLM-4-AirX', 0.9)

'"欧式空间"这个概念可以从两个不同的角度来解释：数学角度和室内设计角度。\n\n**数学角度：**\n\n在数学领域，欧式空间通常指的是欧几里得空间，这是一类特殊的度量空间。其名称来源于古希腊数学家欧几里得，他在公元前300年左右创立了欧几里得几何，这是一种研究平面和空间中点、线、面之间关系的几何学。\n\n欧几里得空间具有以下特点：\n\n1. **度量性质**：它定义了一种度量，即欧几里得距离，用来测量两点之间的距离。这个距离相当于我们日常生活中的直线距离。\n\n2. **向量空间**：在欧几里得空间中，每个点都可以看作是一个向量，这些向量构成了一个向量空间。\n\n3. **内积与正交性**：欧几里得空间中定义了内积（点积），它可以帮助我们确定向量的长度和两个向量之间的夹角。\n\n4. **维度**：最初，欧几里得空间被定义为二维或三维空间，但这个概念可以扩展到n维空间，即包含任意数量维度的空间。\n\n**室内设计角度：**\n\n在室内设计领域，欧式空间通常指的是采用欧洲传统风格的室内设计。这种设计风格强调对称、比例和谐调，以及精致的细节和装饰。\n\n以下是欧式空间的一些特点：\n\n1. **风格元素**：欧式设计风格通常分为新古典和古典传统风格两种，包含了许多高精度的模型元素，如装饰性的柱子、雕塑、壁画等。\n\n2. **空间布局**：在欧式空间设计中，空间的布局通常遵循对称和平衡的原则，房间中家具的摆放和尺寸都有严格的比例。\n\n3. **装饰细节**：装饰细节精致，常用到的有雕刻、线脚、复杂的纹饰等，展现出一种奢华和高级感。\n\n4. **色彩与材质**：在色彩上，欧式设计喜欢使用暖色调，以及丰富的纹理和材质，如丝绸、皮革和木材等。\n\n这两个角度下的“欧式空间”概念完全不同，但都是在其各自领域内非常重要的概念。在数学中，它是一个基础的几何和空间概念；在室内设计中，它代表了一种流行的装饰风格。'

In [5]:
### 
get_completion("今天星期几？", 'GLM-4-0520', 0.9)

'今天是星期三。不过，我需要指出的是，由于我无法访问实时日期信息，这个回答是基于您可能需要的通用信息。若要获取今天的实际星期信息，建议查看电子设备的日历或搜索在线日历服务。'

设计高效 Prompt 的两个关键原则：编写清晰、具体的指令和给予模型充足思考时间。

#### Prompt 设计的原则及使用技巧
##### 1 原则一：编写清晰、具体的指令
 Prompt 需要清晰明确地表达需求，提供充足上下文，使语言模型能够准确理解我们的意图。

In [13]:
get_completion("请介绍一下欧式空间？", 'GLM-4-AirX', 0.9)

'"欧式空间"这个词在不同的领域有不同的含义。在数学领域，它通常指的是欧几里得空间，这是一种特殊的几何空间。在室内设计领域，则通常指的是具有欧洲风格的装饰和设计理念的空间。\n\n首先，从数学的角度来介绍欧式空间：\n\n欧几里得空间，也称为欧式空间，是以古希腊数学家欧几里得的名字命名的。这种空间满足欧几里得几何的公理，是最常见的几何空间之一。在欧几里得空间中，点之间的距离遵循勾股定理，即直角三角形的两条直角边的平方和等于斜边的平方。在二维空间中，这表现为平面几何；在三维空间中，表现为立体几何。这种空间可以被推广到任意有限维度，成为n维欧几里得空间。而在更高维度的数学空间中，如果满足某些特定的条件（如内积的定义），也可以被视为欧式空间的一种推广，比如希尔伯特空间。\n\n在室内设计的语境下，欧式空间通常指的是采用欧洲古典或新古典主义设计风格的室内环境。以下是这个概念的一些特点：\n\n1. 设计风格：欧式空间强调对称、比例和平衡，常用古典元素如雕花、柱子和壁画来营造氛围。\n2. 色彩与材质：通常使用暖色调和丰富的纹理，如丝绸、皮革和实木，来增加空间的质感和温暖感。\n3. 装饰细节：装饰品和家具设计复杂、精致，如欧式风格的镜子、烛台和工艺品。\n4. 实用性与美观性的结合：尽管装饰华丽，但欧式空间也强调功能性，如在有限的空间内通过巧妙的设计达到最大化的利用。\n\n在数学和设计两个领域，欧式空间的概念虽然不同，但都强调某种规则性和秩序感，无论是数学空间中的几何规则，还是设计空间中的审美规则。'

In [14]:
get_completion("在数学角度和室内设计角度下，解释欧式空间的概念和特点。", 'GLM-4-AirX', 0.9)

'从数学角度和室内设计角度来解释欧式空间的概念和特点，我们可以从以下几个方面来探讨：\n\n### 数学角度：\n\n1. **概念**：\n   欧式空间在数学中通常指的是欧几里得空间，这是一种几何空间，其特点是满足欧几里得几何的公理体系。在三维空间中，欧式空间指的是笛卡尔坐标系中的点集，这些点通过欧几里得距离（即直角三角形的勾股定理）来度量距离。\n\n2. **特点**：\n   - **距离度量**：在欧式空间中，两点之间的距离是通过欧几里得距离公式计算的，即 \\(d(p, q) = \\sqrt{(q_1 - p_1)^2 + (q_2 - p_2)^2 + (q_3 - p_3)^2}\\)。\n   - **直线性质**：欧式空间中的直线是唯一的，两点确定一条直线。\n   - **平行线性质**：在欧式空间中，平行线永不相交。\n   - **角的度量**：在欧式空间中，角度是可以通过内积来度量的。\n\n### 室内设计角度：\n\n1. **概念**：\n   欧式空间在室内设计领域中指的是采用欧洲传统装饰风格的空间设计，这种风格强调对称、奢华和浪漫，同时融合了现代设计元素，形成了一种简约而富有内涵的设计风格。\n\n2. **特点**：\n   - **对称性**：欧式空间设计强调对称性，无论是家具的摆放还是装饰元素的布局，都力求达到平衡和和谐。\n   - **奢华感**：通过使用高档材料和精美的装饰品，如水晶灯、大理石地砖等，来营造出奢华的氛围。\n   - **浪漫元素**：欧式设计中常常融入浪漫的元素，如铁艺装饰、浪漫的灯光设计等。\n   - **简化线条**：简约欧式风格简化了传统欧式风格中的复杂线条和装饰，使得空间更加实用和现代。\n   - **色彩运用**：颜色从传统的深色调转变为明亮轻快的色彩，如象牙白色、浅黄色等，以增强空间的明亮度和舒适感。\n   - **装饰元素**：典型的欧式元素如罗马柱、壁炉、白色木格等，为空间增添独特的欧洲风味。\n\n综合来说，欧式空间在数学上是一个具有明确几何特性的空间，而在室内设计中则是一种融合了传统和现代元素，追求奢华和浪漫的设计风格。'

1. 使用分隔符清晰地表示输入的不同部分
2. 寻求结构化的输出
3. 要求模型检查是否满足条件
4. 提供少量示例

In [19]:
prompt = f"""
你的任务是以一致的风格回答问题（注意：文言文和白话的区别）。
<学生>: 请教我何为耐心。
<圣贤>: 天生我材必有用，千金散尽还复来。
<学生>: 请教我何为坚持。
<圣贤>: 故不积跬步，无以至千里；不积小流，无以成江海。骑骥一跃，不能十步；驽马十驾，功在不舍。
<学生>: 请教我何为孝顺。
"""
response = get_completion(prompt=prompt, model='glm-4-0520', temperature=0.99)
print(response)

<圣贤>: 孝者，所以事亲也，敬其身，养其志，导其行，曰孝。不违亲意，顺其心意，承欢膝下，此之谓孝顺。


#### 2 原则二：给模型时间去思考
如何让模型又更多的时间去思考呢？
可以有以下选择：
    1. 逐步引导和分解任务。在语言模型中，通过逐步引导模型展示其推理过程，可以帮助模型建立起对问题的逻辑结构和关联的理解。
    2. 提供详细的背景信息

指定完成任务所需的步骤

In [20]:
text = f"""
在一个迷人的村庄里，兄妹杰克和吉尔出发去一个山顶井里打水。\
他们一边唱着欢乐的歌，一边往上爬，\
然而不幸降临——杰克绊了一块石头，从山上滚了下来，吉尔紧随其后。\
虽然略有些摔伤，但他们还是回到了温馨的家中。\
尽管出了这样的意外，他们的冒险精神依然没有减弱，继续充满愉悦地探索。
"""

prompt = f"""
1-用一句话概括下面用<>括起来的文本。
2-将摘要翻译成英语。
3-在英语摘要中列出每个名称。
4-输出一个 JSON 对象，其中包含以下键：English_summary，num_names。
请使用以下格式：
摘要：<摘要>
翻译：<摘要的翻译>
名称：<英语摘要中的名称列表>
输出 JSON 格式：<带有 English_summary 和 num_names 的 JSON 格式>
Text: <{text}>
"""

response = get_completion(prompt=prompt, model='glm-4-0520', temperature=0.99)
print("response :")
print(response)

response :
1- 用一句话概括：兄妹杰克和吉尔在打水时不幸摔伤，但依然保持着冒险精神。

2- 翻译如下：

摘要翻译：In a charming village, siblings Jack and Jill set out to fetch water from a mountain-top well. While singing joyful songs and climbing up, misfortune struck - Jack tripped over a stone and fell down the mountain, followed by Jill. Although they sustained some bruises, they still returned to their cozy home. Despite such an accident, their spirit of adventure remains unchanged, continuing to explore with joy.

以下是接下来的步骤：

3- 名称列表：
<英语摘要中的名称列表>：Jack, Jill

4- 输出 JSON 格式：

```json
{
  "English_summary": "In a charming village, siblings Jack and Jill set out to fetch water from a mountain-top well. While singing joyful songs and climbing up, misfortune struck - Jack tripped over a stone and fell down the mountain, followed by Jill. Although they sustained some bruises, they still returned to their cozy home. Despite such an accident, their spirit of adventure remains unchanged, continuing to explore with joy.",
  "num_names":

指导模型在下结论之前找出一个自己的解法

In [21]:
prompt = f"""
请判断学生的解决方案是否正确，请通过如下步骤解决这个问题：
步骤：
首先，自己解决问题。
然后将您的解决方案与学生的解决方案进行比较，对比计算得到的总费用与学生计算的总费用是否一致，
并评估学生的解决方案是否正确。
在自己完成问题之前，请勿决定学生的解决方案是否正确。
使用以下格式：
问题：问题文本
学生的解决方案：学生的解决方案文本
实际解决方案和步骤：实际解决方案和步骤文本
学生计算的总费用：学生计算得到的总费用
实际计算的总费用：实际计算出的总费用
学生计算的费用和实际计算的费用是否相同：是或否
学生的解决方案和实际解决方案是否相同：是或否
学生的成绩：正确或不正确
问题：
我正在建造一个太阳能发电站，需要帮助计算财务。
- 土地费用为每平方英尺100美元
- 我可以以每平方英尺250美元的价格购买太阳能电池板
- 我已经谈判好了维护合同，每年需要支付固定的10万美元，并额外支付每平方英尺10美元;
作为平方英尺数的函数，首年运营的总费用是多少。
学生的解决方案：
设x为发电站的大小，单位为平方英尺。
费用：
1. 土地费用：100x美元
2. 太阳能电池板费用：250x美元
3. 维护费用：100,000+100x=10万美元+10x美元
总费用：100x美元+250x美元+10万美元+100x美元=450x+10万美元
实际解决方案和步骤：
"""

response = get_completion(prompt=prompt, model='glm-4-0520', temperature=0.99)
print(response)

问题：我正在建造一个太阳能发电站，需要帮助计算财务。
- 土地费用为每平方英尺100美元
- 我可以以每平方英尺250美元的价格购买太阳能电池板
- 我已经谈判好了维护合同，每年需要支付固定的10万美元，并额外支付每平方英尺10美元;
作为平方英尺数的函数，首年运营的总费用是多少。

学生的解决方案：
设x为发电站的大小，单位为平方英尺。
费用：
1. 土地费用：100x美元
2. 太阳能电池板费用：250x美元
3. 维护费用：100,000+100x=10万美元+10x美元
总费用：100x美元+250x美元+10万美元+100x美元=450x+10万美元

实际解决方案和步骤：
首先计算各个部分的费用：
1. 土地费用为每平方英尺100美元，因此土地费用总计为100x美元，其中x是平方英尺的数量。
2. 太阳能电池板的费用为每平方英尺250美元，所以太阳能电池板的总费用为250x美元。
3. 维护费用由两部分组成，一部分是固定的10万美元，另一部分是每平方英尺10美元，因此维护费用总计为100,000 + 10x美元。

现在，将所有费用加在一起得到总费用：
总费用 = 土地费用 + 太阳能电池板费用 + 维护费用
总费用 = 100x + 250x + (100,000 + 10x)
总费用 = 350x + 100,000 + 10x
总费用 = 360x + 100,000

学生计算的总费用：450x + 10万美元
实际计算的总费用：360x + 10万美元
学生计算的费用和实际计算的费用是否相同：否
学生的解决方案和实际解决方案是否相同：否
学生的成绩：不正确
